cloudflared tunnel --url http://localhost:5000

In [ ]:
pip uninstall jwt

In [2]:
from flask import Flask, jsonify
from flask_jwt_extended import JWTManager, get_jwt_identity, jwt_required


app = Flask("app.py")
app.config['JWT_SECRET_KEY'] = '1042ddc714c0fb4544eb905972575812ccf9719610ab2548'  # Change this to your actual secret key
jwt = JWTManager(app)

# In-memory storage for JWT identities
jwt_identities = []

@app.route('/add_identity', methods=['POST'])
@jwt_required()
def add_identity():
    identity = get_jwt_identity()
    if identity not in jwt_identities:
        jwt_identities.append(identity)
    return jsonify({"msg": "Identity added"}), 200

print(jwt_identities)

if __name__ == "__main__":
    app.run()

[]
 * Serving Flask app 'app.py'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [1]:
import requests
import socket

def get_public_ip():
    # Get the public IP address using an external service
    response = requests.get('https://initiated-dramatic-magnificent-chuck.trycloudflare.com')
    return response.text

def update_ddns(hostname, key, ip=None):
    # Construct the DDNS update URL (replace with your provider's update URL)
    update_url = f"http://{hostname}/nic/update?hostname={hostname}&myip={ip or get_public_ip()}&key={key}"

    # Send the GET request to update the DNS record
    response = requests.get(update_url)
    
    if "good" in response.text:
        print(f"Successfully updated {hostname} to {ip or get_public_ip()}")
    else:
        print(f"Failed to update {hostname}. Response: {response.text}")

if __name__ == "__main__":
    hostname = "fraskhamakha.ddns.net"  # Your DDNS hostname
    key = "hfc1egj"  # Your DDNS key/token

    # Get current public IP and update DDNS
    update_ddns(hostname, key)


ConnectionError: HTTPSConnectionPool(host='initiated-dramatic-magnificent-chuck.trycloudflare.com', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000218FEDC1640>: Failed to resolve 'initiated-dramatic-magnificent-chuck.trycloudflare.com' ([Errno 11001] getaddrinfo failed)"))

In [ ]:
import smtplib
from email.message import EmailMessage
import random
import ssl

def send_confirmation_code(receiver_email):
    # Generate a random 6-digit confirmation code
    confirmation_code = random.randint(100000, 999999)
    
    # Set your Gmail account credentials
    sender_email = "identify.fras@gmail.com"
    sender_password = r"S@msun2204"  # Or an App Password if 2-step verification is enabled
    
    # Create the email content
    msg = EmailMessage()
    msg.set_content(f"Your confirmation code is: {confirmation_code}")
    msg['Subject'] = 'Your Confirmation Code'
    msg['From'] = sender_email
    msg['To'] = receiver_email
    
    # Setup the secure SSL context
    context = ssl.create_default_context()

    try:
        # Connect to Gmail's SMTP server
        with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
            server.login(sender_email, sender_password)
            server.send_message(msg)
            print(f"Confirmation code sent to {receiver_email}")
            return confirmation_code
    except Exception as e:
        print(f"Failed to send email: {e}")
        return None

# Example usage
receiver_email = "abhay.a.shinde@nuv.ac.in"
confirmation_code = send_confirmation_code(receiver_email)
if confirmation_code:
    print(f"Code: {confirmation_code}")


In [2]:
import os.path
import base64
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from email.mime.text import MIMEText

# Update the scope to include sending email
SCOPES = ["https://www.googleapis.com/auth/gmail.send"]

def create_message(sender, to, subject, message_text):
    """Create a message for an email."""
    message = MIMEText(message_text)
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject
    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
    return {'raw': raw}

def send_message(service, user_id, message):
    """Send an email message."""
    try:
        message = service.users().messages().send(userId=user_id, body=message).execute()
        print(f'Message Id: {message["id"]}')
        return message
    except HttpError as error:
        print(f'An error occurred: {error}')
        print(f'Details: {error.content}')  # Print detailed response
        return None


def main():
    """Shows basic usage of the Gmail API.
    Sends an email to the user.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time.
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        # Call the Gmail API
        service = build("gmail", "v1", credentials=creds)
        
        # Define the email details
        sender = "identify.fras@gmail.com"
        to = "abhay.a.shinde@nuv.ac.in"
        subject = "Test Email"
        message_text = "Hello, this is a test email sent using the Gmail API!"

        # Create the email message
        message = create_message(sender, to, subject, message_text)

        # Send the email
        send_message(service, "me", message)

    except HttpError as error:
        # Handle errors from Gmail API
        print(f"An error occurred: {error}")

if __name__ == "__main__":
    main()


Message Id: 192f0b98bfbd9d15


In [21]:
from flask import Flask, request, jsonify
from flask_bcrypt import Bcrypt
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from Database import Connection

In [22]:
db_connection = Connection()
db_connection.connect()

Database connection established.


In [19]:
query = "insert into credentials(email_id, role) values('abhayshinde444@gmail.com', 'admin')"
db_connection.execute_query(query)

In [23]:
query = """
SELECT 
    ts.timetable_id,
    ts.subject_code,
    s.subject_name,
    ts.start_time,
    ts.end_time,
    ts.room_number
FROM timetable ts
JOIN subjects s ON ts.subject_code = s.subject_code
LEFT JOIN class_subject cs ON ts.class_id = cs.class_id AND ts.subject_code = cs.subject_code
WHERE ts.class_id = %s AND ts.day = %s AND cs.teacher_id = %s
"""
db_connection.execute_query(query, (1, "thursday", 2))

((3,
  'CMP513',
  'Computer Graphics and Image Processing ',
  datetime.timedelta(seconds=50400),
  datetime.timedelta(seconds=57600),
  '303'),)

In [13]:
# Query to retrieve all classes for the student's batch, ordered by year and semester type
class_query = """
    SELECT 
        class.class_id,
        class.year,
        class.semester_type
    FROM 
        class
    WHERE 
        class.batch_id = %s
    ORDER BY 
        class.year DESC, 
        class.semester_type DESC
"""

# Execute the class query
class_data = db_connection.execute_query(class_query, (final_data['batch_id'],))

# Add classes data to final_data in descending order
final_data['classes'] = [
    {
        'class_id': class_row[0],
        'year': class_row[1],
        'semester_type': class_row[2]
    }
    for class_row in class_data
]


((1, 3, 'A'), (2, 3, 'S'), (4, 1, 'A'))


In [ ]:
            SELECT 
                ts.timetable_id,
                ts.subject_code,
                s.subject_name,
                ts.start_time,
                ts.end_time,
                ts.room_number
            FROM timetable ts
            JOIN subjects s ON ts.subject_code = s.subject_code
            LEFT JOIN class_subject cs ON ts.class_id = cs.class_id AND ts.subject_code = cs.subject_code
            WHERE ts.class_id = 1 AND ts.day = "thursday" AND cs.teacher_id = 2

In [19]:
teacher_id = 1
year = 2024
month = 11
date = 9
semester_type = "A"

classes = [
    (2, "CS312", 2,  2026, "B", 4),
    (1, "CMP513", 3,  2026, "B", 4)
]

for row in classes:
    class_id = row[0]
    subject_code = row[1]
    classyear = row[2]
    batch_of = row[3]
    branch = row[4]
    course_duration = row[5]

    if semester_type=="A":
        if (batch_of - classyear + 1) == year:
            print(class_id)
    else:
        if (batch_of - classyear) == year:
            print(class_id)



1
